In [1]:
%run train.py --clearml-run-name working1 --batch-size 128 --test-batch-size 256 --epochs 101 --lr 0.001

ClearML Task: created new task id=7ffdfba8e1e146778856112beca62e8c
2023-09-19 11:28:48,322 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/49f472236d7240f4bd330a30202378ad/experiments/7ffdfba8e1e146778856112beca62e8c/output/log


  0%|          | 0/101 [00:00<?, ?it/s]

2023-09-19 11:29:04,854 - clearml.frameworks - INFO - Found existing registered model id=6c68d5510aa94537949f525537df6a22 [/home/quczer/repos/latent-geometry/models/mnist/best_encoder.pt] reusing it.
2023-09-19 11:29:13,466 - clearml.frameworks - INFO - Found existing registered model id=284857ea043048fea1265c8a072d12aa [/home/quczer/repos/latent-geometry/models/mnist/best_decoder.pt] reusing it.


 17%|█▋        | 17/101 [03:05<13:13,  9.45s/it]